In [ ]:
# Import libraries
import pandas as pd
import zipfile
import warnings

warnings.filterwarnings("ignore")

In [53]:
# Load the dataset
!kaggle datasets download hmavrodiev/london-bike-sharing-dataset -p "../data/raw"

with zipfile.ZipFile("../data/raw/london-bike-sharing-dataset.zip", "r") as file:
    file.extractall("../data/raw")

df = pd.read_csv("../data/raw/london_merged.csv")
df.head()

/Users/sezgi/data_analysis/london_bike_analysis/venv_bike/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Dataset URL: https://www.kaggle.com/datasets/hmavrodiev/london-bike-sharing-dataset
License(s): other
london-bike-sharing-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [54]:
# Basic information of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


In [55]:
# Handle duplicate values
print("Duplicate value count:", df.duplicated().sum())

Duplicate value count: 0


In [56]:
# Handle missing values
print(f"Missing Value count: \n{df.isnull().sum()}")

Missing Value count: 
timestamp       0
cnt             0
t1              0
t2              0
hum             0
wind_speed      0
weather_code    0
is_holiday      0
is_weekend      0
season          0
dtype: int64


In [57]:
# Rename the column names
new_cols_dict ={
    'timestamp':'time',
    'cnt':'count', 
    't1':'temp_real_C',
    't2':'temp_feels_like_C',
    'hum':'humidity_ratio',
    'wind_speed':'wind_speed_kph',
    'weather_code':'weather',
    'is_holiday':'is_holiday',
    'is_weekend':'is_weekend',
    'season':'season'
}

df.rename(new_cols_dict, axis=1, inplace=True)

In [58]:
# Change the humidity values to ratio
df.humidity_ratio = df.humidity_ratio / 100
print(df.humidity_ratio)

0        0.930
1        0.930
2        0.965
3        1.000
4        0.930
         ...  
17409    0.810
17410    0.810
17411    0.785
17412    0.760
17413    0.760
Name: humidity_ratio, Length: 17414, dtype: float64


In [ ]:
# Rename the season values
season_dict = {
    '0.0':'spring',
    '1.0':'summer',
    '2.0':'autumn',
    '3.0':'winter'
}

df.season = df.season.astype('str')
df.season = df.season.map(season_dict)

In [60]:
#Rename the weather values
weather_dict = {
    '1.0':'Clear',
    '2.0':'Scattered clouds',
    '3.0':'Broken clouds',
    '4.0':'Cloudy',
    '7.0':'Rain',
    '10.0':'Rain with thunderstorm',
    '26.0':'Snowfall'
}

df.weather = df.weather.astype('str')
df.weather = df.weather.map(weather_dict)

# Check the data types of the columns
df.dtypes



time                  object
count                  int64
temp_real_C          float64
temp_feels_like_C    float64
humidity_ratio       float64
wind_speed_kph       float64
weather               object
is_holiday           float64
is_weekend           float64
season                object
dtype: object

In [61]:
# Check the dataframe
df.head()

,time,count,temp_real_C,temp_feels_like_C,humidity_ratio,wind_speed_kph,weather,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,Broken clouds,0.0,1.0,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,Clear,0.0,1.0,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,Clear,0.0,1.0,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,Clear,0.0,1.0,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,Clear,0.0,1.0,winter


In [62]:
# Save the cleaned dataset in Excel format
load_path = "../data/processed/"
df.to_excel(load_path + "cleaned_london_bikes.xlsx", sheet_name="LondonBikes")